# Tutorial: Cognitive Service Vision Image Composition using Python

This is a tutorial about using Cognitive Service Vision Image Composition.

Cognitive Service Vision Image Composition are a collection of tools to pre-process images before running product recognition, so that better product recognition and planogram compliance results can be achieved.

Currently, image composition feature are available in **EastUS**, **West US2**, and **West Europe** regions.

Please create a computer vision resource on Azure portal, in **EastUS**, **West US2**, or **West Europe** region, if you don't already have one. You can use [Multi-service resource](https://learn.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Canomaly-detector%2Clanguage-service%2Ccomputer-vision%2Cwindows) as well. 

![check resources/create_cv_resource.png if pic does not show up](./resources/create_cv_resource.png)

## Install the python samples package

Install the sample code including utility code helping you use Python to run image composition in this tutorial.

In [ ]:
pip install cognitive-service-vision-product-recogntion-python-samples

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

from cognitive_service_vision_model_customization_python_samples import ResourceType
from cognitive_service_vision_model_customization_python_samples.clients import ImageCompositionClient
from cognitive_service_vision_model_customization_python_samples.models import ImageStitchingRequest

## Credentials

Resource name and resource key are needed for accessing the service, which you can find here:

![check resources/credentials.png if pic does not show up](./resources/credentials.png)

If you are using a multi-service resource, the endpoint can be found here:

![check resources/multi_service_endpoint.png if pic does not show up](./resources/multi_service_endpoint.png)

In [ ]:
# Resource and key
resource_type = ResourceType.SINGLE_SERVICE_RESOURCE # or ResourceType.MULTI_SERVICE_RESOURCE

resource_name = None
multi_service_endpoint = None

if resource_type == ResourceType.SINGLE_SERVICE_RESOURCE:
    resource_name = '{specify_your_resource_name}'
    assert resource_name
else:
    multi_service_endpoint = '{specify_your_service_endpoint}'
    assert multi_service_endpoint

resource_key = '{specify_your_resource_key}'

## Run image stitching

When your shelf is very long that a single camera shot cannot capture the entire length/height of the shelf, use image stitching tool to stitching all of them into one.

### Prepare image urls for running image stitching

Image stitching tool require a list of accessible image urls as input to stitch the image.

In [ ]:
image_urls = [
    "https://example.com/image1.jpg",
    "https://example.com/image2.jpg"
]

### Upload local images to Azure Storage Blob Container and get SAS URLs (Optional)

If you have images locally, consider putting them into a Azure Storage Account Blob Container and get the SAS url for each of the images.

Below is the example code to upload images for stitching: 

In [ ]:
import os
from datetime import datetime, timedelta
from azure.storage.blob import BlobServiceClient, BlobSasPermissions, generate_blob_sas

# Replace these values with your Azure Storage Account credentials
connection_string = "{your_connection_string}"
container_name = "{your_container_name}"

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Create a new container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
container_client.create_container(exist_ok=True)

# Upload images to the container
image_directory = "{path/to/your/images}"
image_files = os.listdir(image_directory)

for image_file in image_files:
    blob_client = container_client.get_blob_client(image_file)
    with open(os.path.join(image_directory, image_file), "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
        print(f"{image_file} uploaded to {container_name} container")

# Get list of SAS URLs
image_urls = []
expiry_time = datetime.utcnow() + timedelta(hours=1)

for image_file in image_files:
    blob_client = container_client.get_blob_client(image_file)
    sas_token = generate_blob_sas(
        account_name=blob_client.account_name,
        container_name=blob_client.container_name,
        blob_name=blob_client.blob_name,
        snapshot=None,
        account_key=blob_client.credential.account_key,
        permission=BlobSasPermissions(read=True),
        expiry=expiry_time
    )

    sas_url = f"{blob_client.url}?{sas_token}"
    image_urls.append(sas_url)

## Run image stitching

In [ ]:
client = ImageCompositionClient(resource_type, resource_name, multi_service_endpoint, resource_key)
stitching_request = ImageStitchingRequest(images=image_urls)
response = client.stitch_images(request=stitching_request)

# Display stitched image
display(Image.open(io.BytesIO(response)))

## Run image recitfication

With image rectification, product recognition and planogram matching results will be better.

The rectification control points can be easily selected using the GUI-based selection tool under `cognitive_service_vision_model_customization_python_samples/tools/select_rectification_control_points.py`.

For the best result, you would want to select the four corners of the shelf in clock-wise order, the output from the selection tool can then be used as input for the image rectification example below:

In [ ]:
from cognitive_service_vision_model_customization_python_samples.models import (
    ImageRectificationRequest,
    ImageRectificationControlPoints,
    NormalizedCoordinate
)
from IPython.display import display

# Accessible image url for image to be rectified, if you have images locally, can also follow the image upload steps above
image_url = "https://example.com/image1.jpg"

control_points = ImageRectificationControlPoints(
    top_left=NormalizedCoordinate(x=0.1, y=0.1),
    top_right=NormalizedCoordinate(x=0.9, y=0.1),
    bottom_left=NormalizedCoordinate(x=0.1, y=0.9),
    bottom_right=NormalizedCoordinate(x=0.9, y=0.9)
)

rectify_request = ImageRectificationRequest(url=image_url, control_points=control_points)
response = client.rectify_image(request=rectify_request)
display(Image.open(io.BytesIO(response)))

## Next steps

With the stitched and rectified image, you can now send the image for product recognition, please refer to the tutorial [here](./cognitive_service_vision_product_recognition.ipynb)